In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#设置seaborn的主题样式为白色网格,另外还有darkgrid(默认)/dark/white/ticks.
sns.set_style(style="whitegrid")

In [ ]:
X = pd.read_csv('xxxx.csv')

In [ ]:
#数据标准化 
from sklearn import preprocessing 
x_scaled = preprocessing.scale(X)
x_scaled = pd.DataFrame(x_scaled,columns=X.columns)

In [2]:
 #Kmeans 寻找最优解

#当K值设置不同时，聚类效果会存在较大的差别，因此在实际应用中需要依据聚类效果，对K值进行优化。执行如下代码，通过轮廓系数(Calinski-Harabasz分数值)对不同的K值进行评估

In [ ]:
#对不同的K值进行计算，筛选最优的K值
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
#kMeans 算法实例化，将其设置为K=range(2,14)
d = {}
fig_reduced_data = plt.figure(figsize=(12,12))
for k in range(2,14):
    est = KMeans(n_clusters=k,random_state=111)
    #作用到标准化后的数据集上
    y_pred = est.fit_predict(x_scaled)
    #距离越来越小，效果越来越好
    score = metrics.calinski_harabasz_score(x_scaled,y_pred)
    d.update({k:score})
    print('calinski_harabasz_score with k={0} is {1}'.format(k,score))

In [ ]:
x = []
y = []
for  k,score in d.items():
    x.append(k)
    y.append(score)

plt.plot(x,y)
plt.xlabel('k value')
plt.ylabel('calinski_harabasz_score')

In [ ]:
#数据降维
#使用PCA进行数据降维
from sklearn.decomposition import PCA
#此外的主成分维度我们人为设定为3，对于属性较少的数据集，属于常规会选择的维度数，后面也会看
pca = PCA(n_components=3)
x_pca = pca.fit_transform(x_scaled)
x_pca_frame = pd.DataFrame(x_pca,columns=['pca_1','pca_2','pca_3'])
x_pca_frame.head()

In [ ]:
#降维指标与原始指标的关联关系
#这三个指标与原始字段的系数可以通过pca的components_属性获取
#绘制主成分与原数据的热图，更直观的反应他们之间的关系
plt.matshow(pac.components_,cmap='plasma')
plt.xticks(range(7),X.columns,fontproperties = 'SimHei')
#y轴显示三个主成分
plt.yticks([0,1,2],['pca_1','pca_2','pca_3'])
plt.colorbar()
#显示图像
plt.show()

从热力图中，我们可以看出每个原始数据和主成分之间的关系，暖色代表正相关，冷色代表负相关，数值的绝对值越大代表相关性越强。至此，我们可以得出，三个主成分基本覆盖了所有的原数据。下面我们再次进行K值寻优操作

In [ ]:
#对不同的K值进行计算，筛选最优的K值

###这里注意 是采用降维之后的数据，重新进行K值的判断！！！！！！！

from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
#kMeans 算法实例化，将其设置为K=range(2,14)
d = {}
fig_reduced_data = plt.figure(figsize=(12,12))
for k in range(2,14):
    est = KMeans(n_clusters=k,random_state=111)
    #作用到标准化后的数据集上
    y_pred = est.fit_predict(x_scaled)
    #距离越来越小，效果越来越好
    score = metrics.calinski_harabasz_score(x_pca_frame,y_pred)
    d.update({k:score})
    print('calinski_harabasz_score with k={0} is {1}'.format(k,score))

In [ ]:
x = []
y = []
for  k,score in d.items():
    x.append(k)
    y.append(score)

plt.plot(x,y)
plt.xlabel('k value')
plt.ylabel('calinski_harabasz_score')

可以发现，进行PCA数据降维之后，聚类效果更好；另外从聚类结果图片可以看出，对于这个实验，当K为2的时候，聚类效果较好
这里有一点是需要注意的，当PCA进行降维的时候，如果N =2 或者 N =3 ，他们最后的结果一样，都是K为2效果最佳，这时候就是看他们的score
他们的score越小，效果则越好

In [ ]:
#K-means聚类建模
#运行Kmean聚类算法
from sklearn.cluster import KMeans
#此处指定K= 12
est = KMeans(n_clusters=12)
est.fit(x_pca)
kmeans_clustering_labels = pd.DataFrame(est.labels_,columns=['cluster'])
#将聚类结果与降维特征数据进行拼接
x_pca_frame = pd.concat([x_pca_frame,kmeans_clustering_labels],axis= 1)
x_pca_frame.head(5)

In [ ]:
#删除噪音点
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

x.index = x_pca_frame.index
x_full = pd.concat([x,x_pca_frame],axis=1)
grouped = x_full.groupby('cluster')
result_data = pd.DataFrame()
for name,group in grouped:
    print(name,group['pca_1'].count())
    desp = group[['pca_1','pca_2','pca_3']].describe()
    for att in ['pca_1','pca_2','pca_3']:
        lower25 = desp.loc['25%',att]
        upper75 = desp.loc['75%',att]
        IQR = upper75 - lower25
        min_value = lower25 - 1.5 * IQR
        max_value = upper75 + 1.5 * IQR
        group = group[(group[att] > min_value) & (group[att] < max_value)]
    result_data = pd.concat([result_data,group],axis=0)
    print(name,group['pca_1'].count())
#分别列出 筛选前后每个特征的数量
print('remanin sample:',result_data['pca_1'].count())

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import glog as log
 
from sklearn.cluster import DBSCAN  # 进行DBSCAN聚类
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score ,calinski_harabasz_score,davies_bouldin_score # 计算 轮廓系数，CH 指标，DBI 
 
 
# 定义一个进行DBSCAN的函数
def DBSCAN_Cluster(embedding_image_feats):
    """
    dbscan cluster
    :param embedding_image_feats:  # 比如形状是（9434,4）表示9434个像素点
    :return:
    """
    db = DBSCAN(eps=0.35, min_samples=600)
    try:
        features = StandardScaler().fit_transform(embedding_image_feats)  # 将特征进行归一化
        db.fit(features)
    except Exception as err:
        log.error(err)   
        ret = {
            'origin_features': None,
            'cluster_nums': 0,
            'db_labels': None,
            'cluster_center': None
            }
        return ret
 
    db_labels = db.labels_                  # 获取聚类之后没一个样本的类别标签
    unique_labels = np.unique(db_labels)    # 获取唯一的类别
 
    num_clusters = len(unique_labels)
    cluster_centers = db.components_
 
    ret = {
            'origin_features': features,      #(9434,4)
            'cluster_nums': num_clusters,     # 5  它是一个标量，表示5类，包含背景
            'db_labels': db_labels,           #(9434,)
            'unique_labels': unique_labels,   #(5,)
            'cluster_center': cluster_centers #(6425,4)
        }
 
    return ret
 
# 画出聚类之后的结果
def plot_dbscan_result(features,db_labels,unique_labels,num_clusters):
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    for k, color in zip(unique_labels, colors):
        if k == -1:
           color = 'k'  # 黑色的，这代表噪声点
 
        index=np.where(db_labels==k)   #  获取每一个类别的索引位置
        x=features[index]
 
        plt.plot(x[:, 0], x[:, 1], 'o', markerfacecolor=color,markeredgecolor='k', markersize=6)
 
    plt.title('Estimated number of clusters: %d' % num_clusters)
    plt.show()
 
 
if __name__=='__main__':
    embedding_features=np.load("./tools/features_logits/lane_embedding_feats.npy")  # 导入数据，数据格式为（samples，）
 
    ret=DBSCAN_Cluster(embedding_features)  # 进行 DBSCAN聚类
 
    plot_dbscan_result(ret['origin_features'],ret['db_labels'],ret['unique_labels'],ret['cluster_nums']) # 展示聚类之后的结果
 
    
    s1=silhouette_score(embedding_features, ret['db_labels'], metric='euclidean') # 计算轮廓系数
    s2=calinski_harabasz_score(embedding_features,ret['db_labels']) # 计算CH score
    s3=davies_bouldin_score(embedding_features,ret['db_labels'])    # 计算 DBI
 
    print(s1)
    print(s2)
    print(s3)
 

FileNotFoundError: [Errno 2] No such file or directory: './tools/features_logits/lane_embedding_feats.npy'